In [4]:
import gdown
import pandas as pd

# Configure display settings to fully display the DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Create a class for Mamdani Fuzzy Inference System
class FuzzyControlSystem:
    def __init__(self, num_input_mfs=13, num_output_mfs=23, velocity_limit=120, distance_limit=1000, pedal_limit=45):
        self.num_input_mfs = num_input_mfs
        self.num_output_mfs = num_output_mfs
        self.velocity_limit = velocity_limit
        self.distance_limit = distance_limit
        self.pedal_limit = pedal_limit
        self.rules_data = self.download_data()

    def download_data(self):
        # Download the dataset using gdown from a specified Google Drive URL
        dataset_url = 'https://drive.google.com/uc?id=1dlvS7Kb-0pYXP3_719yulbF6JG8PtU57'
        dataset_path = 'Updated_Fuzzy_Rules.xlsx'
        gdown.download(dataset_url, dataset_path, quiet=False)
        return pd.read_excel(dataset_path)

    def calculate_mf_parameters(self, num_mfs, max_val):
        # Calculate membership function parameters
        increment = max_val / (num_mfs - 1)
        mf_params = []
        for i in range(num_mfs):
            if i == 0:
                bounds = [0, 0, increment]
            elif i == num_mfs - 1:
                bounds = [(i-1) * increment, i * increment, i * increment]
            else:
                bounds = [(i-1) * increment, i * increment, (i+1) * increment]
            mf_params.append(bounds)
        return mf_params

    def triangular_membership(self, value, params):
        # Define a triangular membership function
        low, peak, high = params
        if low <= value <= peak:
            return (value - low) / (peak - low)
        elif peak < value <= high:
            return (high - value) / (high - peak)
        return 0.0

    def process_input_values(self, velocity, distance):
        # Process the velocity and distance through their membership functions
        velocity_params = self.calculate_mf_parameters(self.num_input_mfs, self.velocity_limit)
        distance_params = self.calculate_mf_parameters(self.num_input_mfs, self.distance_limit)
        membership_velocity = [self.triangular_membership(velocity, params) for params in velocity_params]
        membership_distance = [self.triangular_membership(distance, params) for params in distance_params]
        return membership_velocity, membership_distance

    def execute_rules(self, vel_mf, dist_mf):
        # Execute fuzzy logic rules
        self.rules_data['MF_Velocity'] = [vel_mf[int(idx)-1] for idx in self.rules_data['Velocity Index']]
        self.rules_data['MF_Distance'] = [dist_mf[int(idx)-1] for idx in self.rules_data['Distance Index']]
        self.rules_data['Rule_Result'] = self.rules_data.apply(lambda row: min(row['MF_Velocity'], row['MF_Distance']), axis=1)

    def calculate_output(self, velocity, distance):
        # Calculate output from the fuzzy system
        vel_mf, dist_mf = self.process_input_values(velocity, distance)
        self.execute_rules(vel_mf, dist_mf)
        self.update_output_centers()
        total_product = (self.rules_data['Rule_Result'] * self.rules_data['Center']).sum()
        sum_products = self.rules_data['Rule_Result'].sum()
        if sum_products == 0:
            return 0
        return total_product / sum_products

    def update_output_centers(self):
        # Update the center values for the pedal position
        pedal_params = self.calculate_mf_parameters(self.num_output_mfs, self.pedal_limit)
        centers = [((params[1] + params[2]) / 2 if idx != self.num_output_mfs - 1 else params[1]) for idx, params in enumerate(pedal_params)]
        self.rules_data['Center'] = [centers[int(idx)-1] for idx in self.rules_data['Pedal Index']]

# Instantiate the FuzzyControlSystem
fuzzy_system = FuzzyControlSystem()

# Compute the output for specific velocity and distance inputs
output_pedal_position = fuzzy_system.calculate_output(60, 500)
print("Output pedal position:", output_pedal_position)

# Apply styling to the DataFrame for visual presentation
# Apply styling to the DataFrame for visual presentation
styled_data = fuzzy_system.rules_data.style.applymap(lambda x: 'color: white', subset=['Velocity Index', 'Distance Index', 'Pedal Index']) \
                                           .background_gradient(subset=['MF_Velocity', 'MF_Distance', 'Rule_Result', 'Center'], cmap='plasma') \
                                           .set_properties(**{'text-align': 'center'}) \
                                           .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]) \
                                           .hide(axis='index') \
                                           .format(precision=4)
# Display the styled DataFrame
styled_data

# Display the styled DataFrame
styled_data


Downloading...
From: https://drive.google.com/uc?id=1dlvS7Kb-0pYXP3_719yulbF6JG8PtU57
To: /content/Updated_Fuzzy_Rules.xlsx
100%|██████████| 7.75k/7.75k [00:00<00:00, 14.3MB/s]

Output pedal position: 31.704545454545453


Velocity Index,Distance Index,Pedal Index,MF_Velocity,MF_Distance,Rule_Result,Center
1,1,1,0.0000,0.0000,0.0000,1.0227
1,2,2,0.0000,0.0000,0.0000,3.0682
1,3,3,0.0000,0.0000,0.0000,5.1136
1,4,4,0.0000,0.0000,0.0000,7.1591
1,5,5,0.0000,0.0000,0.0000,9.2045
1,6,6,0.0000,0.0000,0.0000,11.2500
1,7,7,0.0000,1.0000,0.0000,13.2955
1,8,8,0.0000,0.0000,0.0000,15.3409
1,9,9,0.0000,0.0000,0.0000,17.3864
1,10,10,0.0000,0.0000,0.0000,19.4318
